# TemporalStyleNet - Exploration Notebook

This notebook demonstrates the core functionality of the TemporalStyleNet project.

**Topics covered:**
1. Loading and preprocessing data
2. Style transfer on single images
3. Video processing with temporal consistency
4. Evaluation metrics
5. Visualization

In [ ]:
# Imports
import sys
sys.path.append('..')

import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import transforms

from src.models.style_transfer import StyleTransferNet
from src.models.temporal_consistency import TemporalConsistencyModule
from src.inference.video_processor import VideoStyleTransfer

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

## 1. Single Image Style Transfer

In [ ]:
# Initialize model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = StyleTransferNet().to(device)
model.eval()

# Image transforms
transform = transforms.Compose([
    transforms.Resize(512),
    transforms.ToTensor(),
])

# Load images (replace with your paths)
content_img = Image.open('../data/videos/sample_frame.jpg').convert('RGB')
style_img = Image.open('../data/styles/starry_night.jpg').convert('RGB')

content_tensor = transform(content_img).unsqueeze(0).to(device)
style_tensor = transform(style_img).unsqueeze(0).to(device)

print(f"Content shape: {content_tensor.shape}")
print(f"Style shape: {style_tensor.shape}")

In [ ]:
# Apply style transfer
with torch.no_grad():
    stylized = model(content_tensor, style_tensor, alpha=1.0)

# Visualize
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].imshow(content_img)
axes[0].set_title('Content')
axes[0].axis('off')

axes[1].imshow(style_img)
axes[1].set_title('Style')
axes[1].axis('off')

stylized_img = stylized.squeeze(0).cpu().clamp(0, 1).permute(1, 2, 0).numpy()
axes[2].imshow(stylized_img)
axes[2].set_title('Stylized')
axes[2].axis('off')

plt.tight_layout()
plt.show()

## 2. Video Processing

In [ ]:
# Initialize video processor
processor = VideoStyleTransfer(
    method='adain',
    device='cuda',
    use_temporal_consistency=True
)

# Process video
stats = processor.process_video(
    input_path='../data/videos/input.mp4',
    style_path='../data/styles/starry_night.jpg',
    output_path='../data/outputs/stylized_notebook.mp4',
    alpha=0.8,
    max_frames=50  # Process first 50 frames for testing
)

print("\nProcessing Statistics:")
for key, value in stats.items():
    print(f"  {key}: {value}")

## 3. Temporal Consistency Analysis

In [ ]:
from src.models.temporal_consistency import compute_temporal_consistency_metrics

# Load video frames
frames, fps, (width, height) = processor.load_video('../data/outputs/stylized_notebook.mp4')

# Convert to tensor
frames_tensor = torch.stack([
    torch.from_numpy(f).permute(2, 0, 1).float() / 255.0
    for f in frames[:50]
])

# Compute metrics
metrics = compute_temporal_consistency_metrics(frames_tensor)

print("\nTemporal Consistency Metrics:")
for metric, value in metrics.items():
    print(f"  {metric}: {value:.4f}")

## 4. Compare Different Alpha Values

In [ ]:
# Test different style strengths
alphas = [0.3, 0.6, 1.0]

fig, axes = plt.subplots(1, len(alphas) + 1, figsize=(20, 5))

# Original
axes[0].imshow(content_img)
axes[0].set_title('Original')
axes[0].axis('off')

# Different alphas
for i, alpha in enumerate(alphas):
    with torch.no_grad():
        stylized = model(content_tensor, style_tensor, alpha=alpha)
    
    stylized_img = stylized.squeeze(0).cpu().clamp(0, 1).permute(1, 2, 0).numpy()
    axes[i + 1].imshow(stylized_img)
    axes[i + 1].set_title(f'α = {alpha}')
    axes[i + 1].axis('off')

plt.tight_layout()
plt.show()

## 5. Performance Benchmarking

In [ ]:
import time

# Benchmark processing speed
num_runs = 10
times = []

for _ in range(num_runs):
    start = time.time()
    
    with torch.no_grad():
        output = model(content_tensor, style_tensor)
    
    if torch.cuda.is_available():
        torch.cuda.synchronize()
    
    elapsed = time.time() - start
    times.append(elapsed)

avg_time = np.mean(times)
fps = 1.0 / avg_time

print(f"\nPerformance Benchmarks:")
print(f"  Average time: {avg_time:.3f}s")
print(f"  FPS: {fps:.2f}")
print(f"  Std Dev: {np.std(times):.3f}s")

## 6. Export for Resume

Key metrics to include in your resume:
- Processing speed (FPS)
- Temporal consistency improvement
- Model parameters
- Multi-GPU speedup (if trained)

In [ ]:
# Calculate model parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print("\nModel Statistics:")
print(f"  Total parameters: {total_params:,}")
print(f"  Trainable parameters: {trainable_params:,}")
print(f"  Processing speed: {fps:.2f} FPS")
print(f"  Temporal stability: {metrics['stability_score']:.3f}")

## Next Steps

1. **Training**: Use `scripts/train.py` to train your own model
2. **Evaluation**: Run comprehensive benchmarks with `scripts/benchmark.py`
3. **Demo**: Launch interactive demo with `demo/app.py`
4. **Production**: Deploy with your favorite framework

For more information, see the [README](../README.md)